In [5]:
import numpy as np
from pymilvus import utility, connections, Collection
import torch

### Connection Setup

In [2]:
alias = "ahcore_milvus_vector_db"
user = "root"
pwd = "taart123!"
port = 19530
connections.connect(alias=alias, host='gorgophone', port=port, user=user, password=pwd)

## Vector Lookup

### Create Reference Vector Set

In [13]:
print(collection.schema)

{'auto_id': True, 'description': 'pathology fomo CLS embedding collection', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'filename', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 250}}, {'name': 'coordinate_x', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'coordinate_y', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'tile_size', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'mpp', 'description': '', 'type': <DataType.FLOAT: 10>}, {'name': 'embedding', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}], 'enable_dynamic_field': False}


In [22]:
# We need to loop through all annotations of a certain type, find the coordinates, filenames, && labels
# We then do a vector lookup for all the filesnames and coordinates, obtaining the vector entries
# Example query lookup
collection = Collection(name="path_fomo_cls_debug_coll", using=alias)
collection.create_index(field_name="embedding", index_params=index)
collection.load()

res = collection.query(
  expr = "filename == 'images/K102490.svs' && coordinate_x > 20000 && coordinate_x < 40000 && coordinate_y == 0",
  output_fields=["filename", "coordinate_x", "coordinate_y"], 
  consistency_level="Strong",
  limit=5,
)
print(res)


[{'coordinate_y': 0, 'id': 450890313421298898, 'filename': 'images/K102490.svs', 'coordinate_x': 22176}, {'coordinate_y': 0, 'id': 450890313421298899, 'filename': 'images/K102490.svs', 'coordinate_x': 33600}, {'coordinate_y': 0, 'id': 450890313421298900, 'filename': 'images/K102490.svs', 'coordinate_x': 33824}, {'coordinate_y': 0, 'id': 450890313421298901, 'filename': 'images/K102490.svs', 'coordinate_x': 34048}, {'coordinate_y': 0, 'id': 450890313421298902, 'filename': 'images/K102490.svs', 'coordinate_x': 34272}]


### Perform search

#### Create Index

In [9]:
index = {
    "index_type": "FLAT",
    "metric_type": "L2",
    "params": {"nlist": 2048},
}
collection = Collection(name="path_fomo_cls_debug_coll", using=alias)
collection.create_index(field_name="embedding", index_params=index)

Status(code=0, message=)

#### Load Collection

In [10]:
collection.load()

#### Setup Search Params

In [11]:
search_params = {"metric_type": "L2"}
